# Demo
This was my process of selecting a scoring method to determine which model evaluates our code 

## Setup stuff

In [1]:
import langchain
import ollama

In [74]:
# establish models 
qwenSmall = 'qwen2.5-coder:0.5b'
wizCoder = 'wizardcoder:33b'


In [70]:
from langchain.prompts import PromptTemplate

oneShot = PromptTemplate(
    input_variables=["method_code"],
    template='''\
You are a java expert. Write a professional Javadoc comment for the following Java method. Be concise but informative.

Example:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Expected Output:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

make sure to only include the java doc comment, and not the method itself.
now write a expertly written javadoc comment for this method:
{method_code}
'''
)


# Test methods

In [11]:
longMethod = """private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
"""

In [59]:
longmethod2 = '''
public BallSeq clone( )
    {{
        assert wellFormed() : "invariant wrong at start of clone()";

        BallSeq result;

        try
        {{
            result = (BallSeq) super.clone( );
        }}
        catch (CloneNotSupportedException e)
        {{
            // This exception should not occur. But if it does, it would probably
            // indicate a programming error that made super.clone unavailable.
            // The most common error would be forgetting the "Implements Cloneable"
            // clause at the start of this class.
            throw new RuntimeException
            ("This class does not implement Cloneable");
        }}
        // This is inspired by the textbook for this class, page 203
        if(result.head == null) return result;

        Node source = result.head;
        Node copyHead = new Node(source.data, null);
        Node copyTail = copyHead;

        while(source.next != null) {{
            if(source == cursor) result.cursor = copyTail;
            if(source == precursor) result.precursor = copyTail;
            if(source == tail) result.tail = source;
            source = source.next;
            copyTail.next = new Node(source.data, null);
            copyTail = copyTail.next;
        }}
        result.head = copyHead;
        if(source == cursor) result.cursor = copyTail;
        if(source == tail) result.tail = copyTail;

        // TODO: Implemented by student.
        // Now do the hard work of cloning the list.
        // See pp 200-204, 235 (3rd ed. pp. 193-197, 228)
        // Setting precursor, cursor and tail correctly is tricky.


        assert wellFormed() : "invariant wrong at end of clone() for this";
        assert result.wellFormed() : "invariant wrong for result of clone() for copy";
        return result;
    }}
'''

In [79]:
shortMethod = """
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
 """

# Score Based System

In [7]:
def score_method_complexity(method_code):
    lines = method_code.strip().splitlines()
    score = 0

    # Base score: length of method
    if len(lines) > 20:
        score += 2
    elif len(lines) > 10:
        score += 1

    # Keywords that indicate control flow or complexity
    keywords = ["for", "while", "if", "else", "switch", "case", "try", "catch", "finally", "return"]
    score += sum(1 for line in lines for kw in keywords if kw in line)

    # Additional indicators of advanced logic
    if "clone" in method_code or "recursion" in method_code.lower():
        score += 2
    if "assert" in method_code:
        score += 1
    if "TODO" in method_code or "textbook" in method_code.lower():
        score += 1

    # Thresholds
    if score >= 8:
        return "super_difficult", score
    elif score >= 4:
        return "complex", score
    else:
        return "simple", score


## Testing Scoring System 

In [14]:
score_method_complexity(longMethod)

('super_difficult', 29)

In [15]:
score_method_complexity(shortMethod)

('simple', 3)

# LLM Based Approach 

In [37]:
from langchain.prompts import PromptTemplate

routePrompt = PromptTemplate(input_variables=["method_code"],
    template='''
    You are a helpful assistant that helps to determine the complexity of a method.
    You are given a method and you need to determine if it is simple, complex or super difficult.
    A method is considered simple if it has a low number of lines and control flow statements.
    A method is considered complex if it has a moderate number of lines and control flow statements.
    A method is considered super difficult if it has a high number of lines and control flow statements.


    you are only allowed to respond in one word simple, complex or super_difficult.
    You are given the following method:
    {method_code}
    '''
)

## testing outputs

In [38]:
from ollama import chat
response = chat(model=qwenSmall, messages=[{'role': 'user', 'content': routePrompt.format(method_code=shortMethod)}])
print(response['message']['content'])

simple


In [43]:
from ollama import chat
response = chat(model=qwenSmall, messages=[{'role': 'user', 'content': routePrompt.format(method_code=longMethod)}])
print(response['message']['content'])

simple


# Some library that I found online that does exactly what im looking for I love python 
this library looks at how many outcomes our code can take the more outcomes the higher the score 

In [ ]:
# gets the cyclomatic complexity of a method
import lizard

def get_java_method_complexity(method_string):
    wrapped = f" {{\n{method_string}\n}}"
    results = lizard.analyze_file.analyze_source_code("Dummy.java", wrapped)
    complexity = results.function_list[0].cyclomatic_complexity
    return complexity

## Testing

In [69]:
get_java_method_complexity(longMethod)

14

In [68]:
get_java_method_complexity(shortMethod)

3

In [67]:
get_java_method_complexity(longmethod2)

9

## Demo

In [77]:
from ollama import chat

complexity = get_java_method_complexity(longMethod)

if complexity > 7:
    model = wizCoder
else:
    model = qwenSmall

print(f"Using model: {model}")
response = chat(model=model, messages=[{'role': 'user', 'content': oneShot.format(method_code=longMethod)}])
print(response['message']['content'])


Using model: wizardcoder:33b
/**
 * This method removes a node from the tree according to the comparison of its string field with the given string 's'.
 * If 'removeOnce' is true and the count of the removed node is greater than 1, it will decrement the count.
 * If the 'val' parameter is not null and doesn't match the count of the node being removed, the node remains unchanged.
 * It updates 'lastRemoved', 'version', and 'numEntries'.
 * If a successor node has been found to replace the one being deleted, it makes appropriate adjustments such as updating its string and count fields.
 * @param r The current root of the tree/subtree.
 * @param s The string to compare with the string field of each node for removal.
 * @param before A reference to the parent of the soon-to-be deleted node, used when replacing a node with another node.
 * @param removeOnce If true and the count of the removed node is greater than 1, decrements the count.
 * @param val If not null and doesn't match the coun

In [80]:
from ollama import chat

complexity = get_java_method_complexity(shortMethod)

if complexity > 7:
    model = wizCoder
else:
    model = qwenSmall

print(f"Using model: {model}")
response = chat(model=model, messages=[{'role': 'user', 'content': oneShot.format(method_code=shortMethod)}])
print(response['message']['content'])


Using model: qwen2.5-coder:0.5b
### javadoc Comment for `connect` Method

The `connect` method takes a `HexPiece` object, which represents a location in the grid of coordinates on an HTML canvas. It first trims any leading or trailing whitespace from the input string and converts it to lowercase before cleaning.

1. **Trimming**: 
   - The method uses `trim()` to remove any leading or trailing whitespace characters.
   - The cleaned string is stored in the variable `cleaned`.

2. **Lowercasing**:
   - The method then converts the entire string to lowercase using `toLowerCase()`, which ensures that all characters are in the same case.

3. **Cleaning and Reversing**:
   - A `StringBuilder` is used to efficiently build the new string by reversing each character of the cleaned string.
   - The `reverse()` method is called on the `StringBuilder`, which reverses the string in place.
   - Characters at even indices are appended to the result, while characters at odd indices are prepended.

4.